In [1]:
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.8 MB/s eta

In [3]:
import os
from typing import Any, List, Mapping, Optional

import pandas as pd

import torch
import torch.nn as nn

from transformers import AutoTokenizer, RobertaModel, RobertaConfig

from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.chat_models import ChatOpenAI

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

import chromadb
from chromadb.utils import embedding_functions

# chat model

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/NLP/bio_data/bio_val.csv')
df.head()

,id,label,subject,object,text
0,PMC6993212,VL_Breast cancer(유방암)_1,murine Fprs,myeloid cells,"The Contribution of Chemoattractant GPCRs, For..."
1,PMC6998277,VL_Breast cancer(유방암)_1,cancer cell,HIF-1alpha,The interplay between HIF-1alpha and noncoding...
2,PMC7005297,VL_Breast cancer(유방암)_1,peroxisome proliferator-activated receptor,cancer- fibroblasts,Targeting cancer stem cell pathways for cancer...
3,PMC7017042,VL_Breast cancer(유방암)_1,NGB,ERalpha,Divergent Effects of Daidzein and Its Metaboli...
4,PMC7017255,VL_Breast cancer(유방암)_1,p53,UBE4B,"The Role of E3, E4 Ubiquitin Ligase (UBE4B) in..."


In [6]:
client = chromadb.PersistentClient(path="/content/drive/MyDrive/Colab_Notebooks/NLP/bioDB")
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [7]:
client.list_collections()

[Collection(name=bio_paper), Collection(name=bio_test)]

In [8]:
collection = client.get_collection(name="bio_test", embedding_function=default_ef)

In [9]:
api_key = ""

In [10]:
answer_template = """
qustion : {input}
document : {doc}

document 내용을 활용해서 질문에 대답해줘.
"""

In [11]:
nlg_llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=api_key)

In [12]:
prompt_answer = PromptTemplate(input_variables=['doc', 'input'], template=answer_template)
answer_chain = LLMChain(llm=nlg_llm, prompt=prompt_answer, output_key='answer')

In [14]:
user_input = input()

murine Fprs이 뭐야?


In [ ]:
doc = collection.query(
    query_texts=user_input,
    n_results=10,
)

In [18]:
response = answer_chain({"doc": doc, "input": user_input})

In [19]:
display(response['answer'])

'murine Fprs은 murine Fpr (Formylpeptide receptor)의 복수형으로, 쥐에서 발견되는 Formylpeptide receptor들을 가리킵니다. Formylpeptide receptor는 세포의 이동과 염증, 면역 반응, 암 진행 등 다양한 생리적 과정에 관여하는 chemoattractant GPCR (G-protein coupled receptor)입니다. murine Fprs는 다양한 세포 유형뿐만 아니라 고립된 세포에서도 발현되는 것으로 알려져 있습니다. 이들 수용체는 다양한 구조적으로 관련 없는 화학적 신호 분자와 상호 작용할 수 있는 독특한 능력을 가지고 있어 질병의 발생, 진행 및 해결에 참여할 수 있습니다. murine Fprs 중 하나인 Fpr2와 그 염기 아미노산 펩타이드인 Cathelicidin-related antimicrobial peptide (CRAMP)는 대장 점막의 가정상태와 염증 및 관련 종양 형성에서 중요한 매개체로 확인되었습니다. 따라서 murine Fprs는 질병 치료를 위한 잠재적인 대상 분자로 간주될 수 있습니다.'